In [1]:
import numpy as np
import pandas as pd

## Firearm Homicide & Suicide Data

Gun-related homicides have captured lots of public attention. Many anti-gun activists have claimed that a mass shooting occurs roughly once every 200 days and has become more and more frequent in recent years. We want to examine firearm homicide data and see if the anti-gun activists' claim are valid or not and if any actions can be taken to prevent such tragedies.

We collected firearm homicide data from the Center for Disease Control and Prevention (CDC, https://www.cdc.gov/), by year, state, age group, and sex. Also, we collected data of all suicide cases by the same factors. We want to aggregate these two files into one table, containing info of death counts of firearm homicides and total homicides by population, year, state, age group, and sex.

## Homicide

In [2]:
#read firearm homicide and all homicide data
df_homicide = pd.read_csv('CDC_Homicide.csv')
df_homicide_all_injury = pd.read_csv('CDC_Homicide_All_Injury.csv')

In [3]:
df_homicide.shape

(52507, 12)

In [4]:
df_homicide_all_injury.shape

(52708, 12)

In [5]:
df_homicide.head()

,Sex,Race,State,Ethnicity,Age Group,First Year,Last Year,Cause of Death,Year,Deaths,Population,Crude Rate
0,Males,All Races,Alabama,Both,00-04 yrs,1999,2017,Homicide/Legal Intervention Firearm,1999.0,P,150513,.
1,Females,All Races,Alabama,Both,00-04 yrs,1999,2017,Homicide/Legal Intervention Firearm,1999.0,P,144427,.
2,NaN,All Races,Alabama,Both,00-04 yrs,1999,2017,Homicide/Legal Intervention Firearm,1999.0,P,294940,.
3,Males,All Races,Alabama,Both,05-09 yrs,1999,2017,Homicide/Legal Intervention Firearm,1999.0,P,161970,.
4,Females,All Races,Alabama,Both,05-09 yrs,1999,2017,Homicide/Legal Intervention Firearm,1999.0,P,153779,.


In [6]:
df_homicide_all_injury.head()

,Sex,Race,State,Ethnicity,Age Group,First Year,Last Year,Cause of Death,Year,Deaths,Population,Crude Rate
0,Males,All Races,Alabama,Both,00-04 yrs,1999,2017,Homicide/Legal Intervention Injury,1999.0,P,150513,.
1,Females,All Races,Alabama,Both,00-04 yrs,1999,2017,Homicide/Legal Intervention Injury,1999.0,P,144427,.
2,NaN,All Races,Alabama,Both,00-04 yrs,1999,2017,Homicide/Legal Intervention Injury,1999.0,16,294940,5.42
3,Males,All Races,Alabama,Both,05-09 yrs,1999,2017,Homicide/Legal Intervention Injury,1999.0,P,161970,.
4,Females,All Races,Alabama,Both,05-09 yrs,1999,2017,Homicide/Legal Intervention Injury,1999.0,P,153779,.


### Data Understanding
We downloaded all data available from the CDC. Due to the limitation of the CDC database, we can only download the data by four keys. We chose Year, State, Age Group, and Sex. The data source is for WISQARS Fatal Injury data operated by the National Center for Health Statistics. WISQARS provides death counts and death rates for the United States and by state, county, age, race, Hispanic ethnicity, sex, leading cause of death, injury intent, and injury mechanism categories. The data is ranged from 1999 to 2017. For the homicide data, there are 52507 rows of data entry with 12 variables. Each row represents total death count result of homicide/legal intervention due to firearms by year, sex, age group, and state. 

The first row represents the death count of males of 0-4yrs old in Alabama. The second row represnts the death count of females of 0-4yrs old in Alabama. The third row represents the total death count for 0-4yrs old in Alabama. 

Some of the important variables are the following:
<ol>
<li><b>Sex:</b> Sex of the deceased.</li> 
<li><b>State:</b> State of the death.</li>
<li><b>Age Group:</b> Age group of the deceased group together by 5 years (0-4 yrs, 5-9 yrs, ...80-84 yrs, 85+ yrs).</li>
<li><b>Year:</b> Year of incidents from 1999-2017.</li>
<li><b>Population:</b> Population by state.</li>
<li><b>Crude Rate:</b> Number of death per 100,000 people.</li>
</ol>

### Data Preparation
<ul>
    <li>We dropped columns with only one value: Race, Ethnicity, First Year, Last Year and Cause of Death.</li>
    <li>The dataset contained a sum row of both genders after every two rows, so we dropped it.</li>
    <li>The number of death was denoted as 'P' if it was zero, so we replaced it.</li>
    <li>The Crude Rate was denoted as '.' if it was zero, so we replaced as well.</li>
    <li>We created new variables: HF stands for homicide firearm, HI stands for homicide injury, SF stands for suicide firearm, SI stands for suicide injury.</li>
</ul>

In [7]:
df_homicide.isna().sum()

Sex               17529
Race                  0
State                 1
Ethnicity             0
Age Group             0
First Year            0
Last Year             0
Cause of Death        0
Year                  1
Deaths                0
Population            0
Crude Rate            0
dtype: int64

In [8]:
df_homicide = df_homicide[df_homicide.isna() == False]
df_homicide.drop(columns=['Race','Ethnicity','First Year','Last Year','Cause of Death'],inplace=True)
df_homicide = df_homicide[df_homicide.isna() == False]
df_homicide['Crude Rate'].replace('.',0,inplace=True)
df_homicide.Deaths.replace('P',0,inplace=True)
df_homicide['Crude Rate'] = df_homicide['Crude Rate'].astype(float)

The dataset of all homicides followed the same pattern. Thus we cleaned it in the same manner. One thing to notice is that since Population is duplicate in both datasets, we dropped the one in this dataset.

In [9]:
df_homicide_all_injury.isna().sum()

Sex               17612
Race                  0
State                 1
Ethnicity             0
Age Group             0
First Year            0
Last Year             0
Cause of Death        0
Year                  1
Deaths                0
Population            0
Crude Rate            0
dtype: int64

In [10]:
df_homicide_all_injury = df_homicide_all_injury[df_homicide_all_injury.isna() == False]
df_homicide_all_injury.drop(['Race','Ethnicity','First Year','Last Year','Cause of Death','Population'],axis=1,inplace=True)
df_homicide_all_injury['Crude Rate'].replace('.',0,inplace=True)
df_homicide_all_injury.Deaths.replace('P',0,inplace=True)
df_homicide_all_injury['Crude Rate'] = df_homicide_all_injury['Crude Rate'].astype(float)

In [11]:
#check
df_homicide.head()

,Sex,State,Age Group,Year,Deaths,Population,Crude Rate
0,Males,Alabama,00-04 yrs,1999.0,0,150513,0.0
1,Females,Alabama,00-04 yrs,1999.0,0,144427,0.0
2,NaN,Alabama,00-04 yrs,1999.0,0,294940,0.0
3,Males,Alabama,05-09 yrs,1999.0,0,161970,0.0
4,Females,Alabama,05-09 yrs,1999.0,0,153779,0.0


In [12]:
#check
df_homicide_all_injury.head()

,Sex,State,Age Group,Year,Deaths,Crude Rate
0,Males,Alabama,00-04 yrs,1999.0,0,0.00
1,Females,Alabama,00-04 yrs,1999.0,0,0.00
2,NaN,Alabama,00-04 yrs,1999.0,16,5.42
3,Males,Alabama,05-09 yrs,1999.0,0,0.00
4,Females,Alabama,05-09 yrs,1999.0,0,0.00


We merged both datasets on keys: Year, State, Age Group, and Sex. As we only wanted to keep records with data from both tables, we used inner join.

In [13]:
df_fu = df_homicide.merge(df_homicide_all_injury, left_on=['Sex','State','Age Group','Year'],right_on=['Sex','State','Age Group','Year'],how='inner')

In [14]:
df_fu.head()

,Sex,State,Age Group,Year,Deaths_x,Population,Crude Rate_x,Deaths_y,Crude Rate_y
0,Males,Alabama,00-04 yrs,1999.0,0,150513,0.0,0,0.00
1,Females,Alabama,00-04 yrs,1999.0,0,144427,0.0,0,0.00
2,NaN,Alabama,00-04 yrs,1999.0,0,294940,0.0,16,5.42
3,Males,Alabama,05-09 yrs,1999.0,0,161970,0.0,0,0.00
4,Females,Alabama,05-09 yrs,1999.0,0,153779,0.0,0,0.00


In [15]:
df_fu.columns

Index(['Sex', 'State', 'Age Group', 'Year', 'Deaths_x', 'Population',
       'Crude Rate_x', 'Deaths_y', 'Crude Rate_y'],
      dtype='object')

In [16]:
#rename the columns
df_fu.rename(index=str,columns = {'Deaths_x':'Deaths HF', 'Crude Rate_x':'Crude Rate HF','Deaths_y':'Deaths HI',\
                                    'Crude Rate_y':'Crude Rate HI'},inplace=True)

## Suicide

After a rough search, we realized that there are more firearm suicides than firearm crimes. Also, people who committed death with a gun had a very slim chance to be saved. This is a sad and shocking fact as when we talked about gun violence, we always associated it with mass shootings or other crimes, not with hurting or killing people themselves. Thus, we would like to explore the data of firearm suicide and see if there are any actions to take from the findings. We collected firearm suicide data from the same database using the same filter. We want to aggregate these two files into one table, containing info of death counts of firearm suicides and total suicides by population, year, state, age group, and sex.

In [17]:
#read firearm suicide and all suicide data
df_suicide = pd.read_csv('CDC_Suicide.csv')
df_all_suicide = pd.read_csv('CDC_Suicide_All_Injury.csv')

In [18]:
#check
df_suicide.head()

,Sex,Race,State,Ethnicity,Age Group,First Year,Last Year,Cause of Death,Year,Deaths,Population,Crude Rate
0,Males,All Races,Alabama,Both,00-04 yrs,1999,2017,Suicide Firearm,1999.0,P,150513,.
1,Females,All Races,Alabama,Both,00-04 yrs,1999,2017,Suicide Firearm,1999.0,P,144427,.
2,NaN,All Races,Alabama,Both,00-04 yrs,1999,2017,Suicide Firearm,1999.0,P,294940,.
3,Males,All Races,Alabama,Both,05-09 yrs,1999,2017,Suicide Firearm,1999.0,P,161970,.
4,Females,All Races,Alabama,Both,05-09 yrs,1999,2017,Suicide Firearm,1999.0,P,153779,.


In [19]:
#check
df_all_suicide.head()

,Sex,Race,State,Ethnicity,Age Group,First Year,Last Year,Cause of Death,Year,Deaths,Population,Crude Rate
0,Males,All Races,Alabama,Both,00-04 yrs,1999,2017,Suicide Injury,1999.0,P,150513,.
1,Females,All Races,Alabama,Both,00-04 yrs,1999,2017,Suicide Injury,1999.0,P,144427,.
2,NaN,All Races,Alabama,Both,00-04 yrs,1999,2017,Suicide Injury,1999.0,P,294940,.
3,Males,All Races,Alabama,Both,05-09 yrs,1999,2017,Suicide Injury,1999.0,P,161970,.
4,Females,All Races,Alabama,Both,05-09 yrs,1999,2017,Suicide Injury,1999.0,P,153779,.


In [20]:
df_all_suicide.shape

(52553, 12)

In [21]:
df_suicide.shape

(52417, 12)

In [22]:
df_suicide.drop(['Race','Ethnicity','First Year','Last Year','Cause of Death'],axis=1,inplace=True)
df_suicide = df_suicide[df_suicide.Sex.isna() == False]
df_suicide.Deaths.replace('P',0,inplace=True)
df_suicide['Crude Rate'].replace('.',0,inplace=True)
df_suicide['Crude Rate'] = df_suicide['Crude Rate'].astype(float)
df_suicide.Year = df_suicide.Year.astype('int')

### Data Preparation

The dataset of all suicides followed the same pattern. Thus we cleaned it with the same manner. One thing to notice is that since Population is duplicate in both datasets, we dropped the one in this dataset.

In [23]:
df_all_suicide.drop(['Race','Ethnicity','First Year','Last Year','Cause of Death','Population'],axis=1,inplace=True)
df_all_suicide = df_all_suicide[df_all_suicide.Sex.isna() == False]
df_all_suicide.Deaths.replace('P',0,inplace=True)
df_all_suicide['Crude Rate'].replace('.',0,inplace=True)
df_all_suicide['Crude Rate'] = df_all_suicide['Crude Rate'].astype(float)
df_all_suicide.Year = df_all_suicide.Year.astype('int')

We merged both datasets on keys: Year, State, Age Group, and Sex. As we only wanted to keep records with data from both tables, we used inner join.

In [26]:
df_full = df_suicide.merge(df_all_suicide, left_on=['Sex','State','Age Group','Year'],right_on=['Sex','State','Age Group','Year'],how='inner')

In [27]:
df_full

,Sex,State,Age Group,Year,Deaths_x,Population,Crude Rate_x,Deaths_y,Crude Rate_y
0,Males,Alabama,00-04 yrs,1999,0,150513,0.00,0,0.00
1,Females,Alabama,00-04 yrs,1999,0,144427,0.00,0,0.00
2,Males,Alabama,05-09 yrs,1999,0,161970,0.00,0,0.00
3,Females,Alabama,05-09 yrs,1999,0,153779,0.00,0,0.00
4,Males,Alabama,10-14 yrs,1999,0,163000,0.00,0,0.00
5,Females,Alabama,10-14 yrs,1999,0,154380,0.00,0,0.00
6,Males,Alabama,15-19 yrs,1999,24,166099,14.45,30,18.06
7,Females,Alabama,15-19 yrs,1999,0,161690,0.00,0,0.00
8,Males,Alabama,20-24 yrs,1999,37,151548,24.41,46,30.35
9,Females,Alabama,20-24 yrs,1999,0,154634,0.00,10,6.47


In [28]:
#rename the columns
df_full.rename(index=str,columns = {'Deaths_x':'Deaths SF', 'Crude Rate_x':'Crude Rate SF','Deaths_y':'Deaths SI',\
                                    'Crude Rate_y':'Crude Rate SI'},inplace=True)

In [29]:
df_full = df_full.merge(df_fu, left_on=['Sex','State','Age Group','Year'],right_on=['Sex','State','Age Group','Year'],how='inner')

Since the data structure for the homicide and suicide merged, data are the same. We can aggregate the data further with all death counts and crude rate for homicide firearm, homicide injury, suicide firearm, and suicide injury by sex, age, year, and state. 

In [30]:
df_full.drop(columns = 'Population_y',inplace=True)

In [31]:
df_full.rename(index=str, columns = {'Population_x':'Population'},inplace=True)

In [32]:
df_full.head(20)

,Sex,State,Age Group,Year,Deaths SF,Population,Crude Rate SF,Deaths SI,Crude Rate SI,Deaths HF,Crude Rate HF,Deaths HI,Crude Rate HI
0,Males,Alabama,00-04 yrs,1999,0,150513,0.00,0,0.00,0,0.00,0,0.00
1,Females,Alabama,00-04 yrs,1999,0,144427,0.00,0,0.00,0,0.00,0,0.00
2,Males,Alabama,05-09 yrs,1999,0,161970,0.00,0,0.00,0,0.00,0,0.00
3,Females,Alabama,05-09 yrs,1999,0,153779,0.00,0,0.00,0,0.00,0,0.00
4,Males,Alabama,10-14 yrs,1999,0,163000,0.00,0,0.00,0,0.00,0,0.00
5,Females,Alabama,10-14 yrs,1999,0,154380,0.00,0,0.00,0,0.00,0,0.00
6,Males,Alabama,15-19 yrs,1999,24,166099,14.45,30,18.06,30,18.06,34,20.47
7,Females,Alabama,15-19 yrs,1999,0,161690,0.00,0,0.00,0,0.00,11,6.80
8,Males,Alabama,20-24 yrs,1999,37,151548,24.41,46,30.35,48,31.67,54,35.63
9,Females,Alabama,20-24 yrs,1999,0,154634,0.00,10,6.47,0,0.00,12,7.76


In [33]:
#reorder the column
df_full = df_full[['Sex','State','Age Group','Year','Population',\
                   'Deaths SF','Crude Rate SF','Deaths SI','Crude Rate SI',\
                   'Deaths HF','Crude Rate HF','Deaths HI','Crude Rate HI']]
df_full.head()

,Sex,State,Age Group,Year,Population,Deaths SF,Crude Rate SF,Deaths SI,Crude Rate SI,Deaths HF,Crude Rate HF,Deaths HI,Crude Rate HI
0,Males,Alabama,00-04 yrs,1999,150513,0,0.0,0,0.0,0,0.0,0,0.0
1,Females,Alabama,00-04 yrs,1999,144427,0,0.0,0,0.0,0,0.0,0,0.0
2,Males,Alabama,05-09 yrs,1999,161970,0,0.0,0,0.0,0,0.0,0,0.0
3,Females,Alabama,05-09 yrs,1999,153779,0,0.0,0,0.0,0,0.0,0,0.0
4,Males,Alabama,10-14 yrs,1999,163000,0,0.0,0,0.0,0,0.0,0,0.0


In [35]:
#save to csv
df_full.to_csv('Suicide_Homicide_Suicide_All.csv')

## Number of Deaths Due To Injury By Firearms

We obtained this dataset that contains the yearly number of deaths due to injury by firearms per 100,000 population (an age-adjusted rate, which accounts for the differences in population composition) in United States with all states from 1999-2017 from https://www.kff.org/other/state-indicator/firearms-death-rate-per-100000/. This dataset aggragated the total number of deaths caused by firearm anually in the US, with numbers in al 50 states and total nationally. We can explore through this data the overall trend from 1999-2017.

In [36]:
df3 = pd.read_excel('Number of Deaths Due to Injury by Firearms.xlsx',index_col='Location')
df3.head()

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Location,,,,,,,,,,,,,,,,,,,
United States,10.3,10.2,10.3,10.5,10.3,10.0,10.3,10.3,10.3,10.3,10.1,10.1,10.2,10.5,10.4,10.3,11.1,11.8,12.0
Alabama,17.7,17.2,16.4,16.1,16.9,14.8,16.0,16.8,17.3,17.3,17.2,16.2,16.0,17.1,17.6,16.9,19.7,21.5,22.9
Alaska,15.8,17.9,14.9,19.9,19.5,17.5,17.5,17.1,18.1,21.0,14.9,20.4,17.5,17.8,19.8,19.2,23.4,23.3,24.5
Arizona,16.3,15.6,16.0,18.0,15.4,15.9,16.1,16.3,15.4,14.4,13.5,14.6,14.7,14.1,14.1,13.5,13.8,15.2,15.8
Arkansas,14.6,15.4,15.3,16.3,15.1,14.7,15.7,15.1,15.1,15.5,16.2,14.4,16.1,16.6,16.8,16.6,17.0,17.8,20.3


In [37]:
df3.shape

(52, 19)

In [38]:
df3.isna().sum()

1999    0
2000    0
2001    0
2002    0
2003    0
2004    0
2005    0
2006    0
2007    0
2008    0
2009    0
2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
dtype: int64

There is no missing value in this dataset.

Below is the summary statistics for number of death caused by firearm per 100,000 population each year from 1999-2017.

In [39]:
df3.describe()

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.00000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,11.253846,10.884615,11.115385,11.463462,11.194231,10.753846,11.05000,10.859615,10.892308,11.136538,10.838462,11.028846,11.067308,11.453846,11.442308,11.425000,12.465385,13.044231,13.513462
std,4.411281,4.200345,4.349459,5.015976,4.634881,4.117941,4.26318,4.082784,4.363792,4.218681,3.865085,3.753865,3.905688,4.032635,4.074808,4.072619,4.693879,4.832708,5.268219
min,3.000000,2.700000,3.000000,2.800000,2.900000,3.100000,2.10000,2.400000,2.500000,3.000000,3.100000,3.200000,3.000000,3.400000,2.600000,2.600000,3.000000,3.400000,2.500000
25%,8.675000,7.950000,8.100000,8.850000,8.050000,7.800000,8.37500,8.000000,8.175000,8.500000,8.425000,8.500000,8.550000,9.075000,8.975000,9.300000,9.750000,10.575000,11.250000
50%,10.750000,10.450000,10.750000,10.950000,11.000000,10.500000,10.80000,10.700000,10.500000,10.650000,10.600000,10.800000,11.000000,11.050000,11.300000,11.400000,11.950000,12.750000,12.800000
75%,13.725000,13.325000,13.850000,14.000000,14.125000,13.100000,13.80000,13.250000,13.800000,13.850000,13.550000,14.125000,14.100000,14.650000,14.150000,14.650000,15.225000,17.175000,16.825000
max,25.500000,22.800000,26.000000,30.300000,26.000000,22.400000,23.80000,20.400000,21.600000,21.000000,18.100000,20.400000,18.900000,20.100000,19.800000,19.200000,23.400000,23.300000,24.500000


Below is the summary statistics for sum of number of death caused by firearm per 100,000 population of United States and each state from 1999-2017.

In [40]:
df3.T.describe()

Location,United States,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,...,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,10.489474,17.452632,18.947368,15.194737,16.031579,8.589474,11.589474,5.147368,9.563158,18.578947,...,9.578947,15.357895,11.000000,11.200000,9.247368,10.831579,9.173684,14.278947,8.636842,16.168421
std,0.546600,1.947753,2.722615,1.159729,1.375198,0.829923,1.080584,0.623938,1.394916,6.372822,...,1.683442,1.013419,0.554777,1.395628,1.342621,0.600974,0.669861,1.491300,1.101116,2.508663
min,10.000000,14.800000,14.900000,13.500000,14.400000,7.400000,10.300000,4.100000,6.600000,8.900000,...,6.200000,14.200000,10.300000,9.400000,6.800000,10.100000,8.300000,12.600000,7.400000,11.000000
25%,10.250000,16.300000,17.500000,14.250000,15.100000,7.850000,10.800000,4.750000,8.800000,13.450000,...,8.800000,14.700000,10.650000,10.000000,8.150000,10.350000,8.750000,13.250000,7.950000,14.850000
50%,10.300000,17.100000,18.100000,15.400000,15.700000,8.500000,11.500000,5.100000,9.100000,20.100000,...,9.700000,15.200000,10.900000,10.700000,9.200000,10.800000,9.000000,14.100000,8.300000,16.700000
75%,10.450000,17.450000,20.150000,15.950000,16.600000,9.300000,12.100000,5.600000,10.600000,23.300000,...,10.250000,15.600000,11.150000,12.350000,10.000000,11.050000,9.500000,14.550000,8.650000,17.550000
max,12.000000,22.900000,24.500000,18.000000,20.300000,9.800000,14.300000,6.400000,12.100000,30.300000,...,13.400000,18.400000,12.400000,14.000000,11.700000,12.100000,11.100000,18.600000,11.400000,20.100000


## Next Step and Limitations of the Datasets

As of now, we have data on the death counts of homicides that are related to firearm, total death counts of homicides, death counts suicides that are related to firearm, and total death counts of suicides. We would be able to compute the percentage of homicides/suicides that are gun related. Furthermore, we would be able to analyze the difference of crude rate by age group to see which age group is more likely to be victimized by firearms, and which year the crude rate is higher. 

However, there's a clear limitation of the data, as we don't have any data on gun ownership. We would like to collect another dataset on annual gun ownership. With the gun ownership data, we would be able to see whether there's a correlation between gun ownership and suicide/homicide caused by firearms. Besides that, all three datasets are aggregated data. We do not have granular information on ethnicity, age, sex, gun type on each incident. 